In [91]:
import pandas as pd
import jellyfish
import numpy as np
from tqdm import tqdm
import re

In [92]:
def no_lastname(lname):
    #blank lastname in csv >>> NaN in Pandas
    return lname=='-' or lname=='0' if isinstance(lname,str) else True

In [93]:
def is_person(title):
    keywords = ['นาย','นาง','นางสาว']
    if title in keywords:
        return True
    return False

In [94]:
def is_person_name(name):
    keywords = ['นาย','นาง','นางสาว','คุณหญิง','น.ส.','ม.ล.','พล.ท.','ร.ท.','พ.ต.ท.','ม.ร.ว.','DR.','MR.','MRS.','ร.ต.ท.','ด.ญ.','ด.ช.','ว่าที่ ร.ต.','รองศาสตราจารย์','ดร.','รศ.','ผศ.']
    for keyword in keywords:
        if not isinstance(name, str):
            print(name)
            break
        if name.find(keyword) == 0:
            return True
    return False

In [95]:
#check if the name is all eng.
def is_eng(name):
    eng_re = re.compile('^([a-zA-Z0-9-_().,\s/&+\'\"])*$')
    if eng_re.match(name):
        return True
    else:
        return False

In [96]:
#drop out spaces and .
def drop_separators(str):
    try:
        return str.replace('.','').replace(' ','')
    except:
        print('cannot process:',str)
        return ''

In [97]:
def drop_common_words(str):
    common_words = ['บริษัท','จำกัด','(ประเทศไทย)','(มหาชน)']
    out = str
    try:
        for word in common_words:
            out = out.replace(word, '')
    except:
        print('cannot process:',str)
        return ''
    return out

In [98]:
def preprocess_name(str):
    return drop_common_words(drop_separators(str)).strip()

In [99]:
#return the id's and the jaro scores for the top k matches of a firm name
#name = firm name to match. The name will be matched against the whole master file
#count = number of top matches
##############################
def get_top_matches(name, master, count=10):
    dist = master['mod_name'].apply(lambda a: jellyfish.jaro_distance(a,name))
    topk = dist.nlargest(count)
    indices = topk.index.values
    ids = master['jrst_id'][indices]
    return ids.values, topk.values

In [100]:
def preprocess_query_file(path):
    try:
        query = pd.read_csv(path)
    except:
        #thai
        query = pd.read_csv(path, encoding='cp874')
        
    #rename idx
    query = query.rename({'เลขทะเบียน':'id','คำนำหน้า':'title','ชื่อ':'name','สกุล':'lname','สัญชาติ':'nat','อาชีพ':'occ','จำนวนหุ้นที่ถือ':'stock'},axis='columns')

    #drop dup
    query = query.drop_duplicates()

    #dropping unused
    query = query.drop(['stock'],axis=1)

    #filter only th. nat
    query = query[query['nat'] == 'TH']

    #select only ones without last name
    query = query[query['lname'].apply(no_lastname)]
    
    #filter out using title
    query = query[~query['title'].apply(is_person)]
    
    #filter ppl out using name
    query = query[~query['name'].apply(is_person_name)]    

    query['mod_name'] = query['name'].apply(preprocess_name)
    
    return query

In [101]:
def preprocess_master_file(path):
    master = pd.read_csv(path)

    master = master.drop_duplicates()

    #drop nan (last element is a nan)
    master = master.dropna(subset=['jrst_nm'])

    #drop unused column
    master = master.drop([
        'source'
    ],
    axis=1)

    #rename column
    master = master.rename({'jrst_nm':'name_2'},axis='columns')

    #define a temporary column containing modified firm names for matching purpose
    master['mod_name'] = master['name_2'].apply(preprocess_name)
    
    return master

In [108]:
#matches two pandas dataframe,
#Query file needs name and mod_name column
def match(query, master, output_prefix):
    merged = pd.merge(query,master,on=['mod_name'],how='inner')

    try:
        #drop unused columns
        merged = merged.drop(['nat', 'occ', 'lname','mod_name'],axis=1)
        
        #rename columns
        merged = merged.rename({'jrst_id':'id_in_master','name_2':'name_in_master'}, axis=1)
    except e:
        pass
    
    #drop duplicates
    merged.drop_duplicates()

    #save
    merged.to_csv('{}_merged.csv'.format(output_prefix))

    #get unmatched names
    unmatched = query[~query['name'].isin(merged['name'])]

    #restart index number from 1
    unmatched = unmatched.reset_index(drop=True)

    #remove matched names
    #edit: Can't -- sometimes there are more than one flavors of the same name where one perfectly matches and others don't.
    #master = master[~master['name_2'].isin(merged['name'])]

    #create columns for similarity
    unmatched['jrst_id'], unmatched['sim_score'] = 0,0.0

    #indices for the newly created columns
    idloc = unmatched.columns.get_loc('jrst_id')
    scoreloc = unmatched.columns.get_loc('sim_score')

    #create empty placeholder for matched data
    partial_match = pd.DataFrame(columns=unmatched.columns)

    size = unmatched.shape[0]
    for i in tqdm(range(unmatched.shape[0])):
        firm_name = unmatched['mod_name'][i]
        #get top k matches
        ids, scores = get_top_matches(firm_name, master)
#         print("{} of {}".format(i, size))
        #add all top k matches to the placeholder
        for j in range(len(ids)):
            #first, modify the original query set
            unmatched.iat[i,idloc] = ids[j]
            unmatched.iat[i,scoreloc] = scores[j]
            #copy the original row to the new dataframe
            partial_match = partial_match.append(unmatched.loc[i])

    partial_match = pd.merge(partial_match, master[['jrst_id','name_2']],on=['jrst_id'],how='left')

    partial_match = partial_match.rename({'jrst_id':'id_in_master','name_2':'name_in_master'}, axis=1)

    try:
        partial_match = partial_match.drop(['nat', 'occ', 'lname', 'mod_name'], axis=1)
    except e:
        pass

    partial_match.to_csv('{}_partial.csv'.format(output_prefix))

In [103]:
import os
output_path = './output/'
input_path = './queryfiles/'
master_filename = './master.csv'

In [104]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [105]:
files = os.listdir(input_path)
files = list(filter(lambda s: '.csv' in s, files))

In [106]:
idx = 0
for file in files: 
    print(input_path+file)
    idx = idx + 1
    
    print('===============================================')
    print('Processing file {} of {}'.format(idx,len(files)))
    print('===============================================')

    prefix = file[:-4]
    query = preprocess_query_file(input_path+file)
    master = preprocess_master_file(master_filename)
    match(query, master, output_path+prefix)

./queryfiles/TS_10-60-108121_007.csv
Processing file 1 of 1
nan
cannot process: nan


  1%|          | 2/295 [00:03<07:47,  1.60s/it]


KeyboardInterrupt: 

## Special Case for SET

In [88]:
set_q = pd.read_csv('./SET_Ownership_Techit.csv')

#rename for matching
set_q = set_q.rename({'holder_name_o':'name'},axis='columns')

#drop blank cols
set_q = set_q.drop(["Unnamed: 2", "Unnamed: 3","Unnamed: 4"], axis='columns')

#drop dups
set_q = set_q.drop_duplicates()

#filter ppl out using name
set_q = set_q[~set_q['name'].apply(is_person_name)]    

#filter out full eng names
set_q = set_q[~set_q['name'].apply(is_eng)]

set_q['mod_name'] = set_q['name'].apply(preprocess_name)

In [107]:
master = preprocess_master_file(master_filename)
match(set_q, master, './output/')

Index(['symbol', 'name', 'mod_name'], dtype='object')

In [109]:
master_filename

'./master.csv'

In [110]:
output_path+prefix

'./output/TS_10-60-108121_007'